In [12]:
def kontsevich(d):
    """
    Returns the number of rational curves of degree d passing through 3d-1 
    general points in P^2.
    These numbers are also the genus zero Gromov-Witten invariants for P^2. 
    
    EXAMPLES::
    
        sage: kontsevich(3)
        12
        sage: kontsevich(4)
        620
        sage: kontsevich(5)
        87304
    """
    if d == 1:
        return 1
    if d == 2:
        return 1
    f = 0
    for i in range(1,d):
        j = d - i
        a = kontsevich(i)
        b = kontsevich(j)
        f = f + i**2*j*(j*binomial(3*d-4,3*i-2)-i*binomial(3*d-4,3*i-1))*a*b
    return f

def bipart(l):
    """
    Needs this to compute genus zero Gromow-Witten invariants for projective 
    spaces.
    EXAMPLES::
        sage: bipart([1,2])  
        [[[2], [1]], [[1, 2], []], [[], [1, 2]], [[1], [2]]]
        sage: len(bipart([1,2]))
        4
        sage: type(bipart([1,2]))
        <type 'list'>
    """
    if len(l) == 0:
        return [[[],[]]]
    elif len(l) == 1:
        k = l[0]
        return [[[],[k]],[[k],[]]]
    else:
        ll = [l[i] for i in range(len(l)-1)]
        R = bipart(ll)
        R1 = [[R[i][0] + [l[-1]],R[i][1]] for i in range(len(R))]
        R2 = [[R[i][0],R[i][1] + [l[-1]]] for i in range(len(R))]
        return R1 + R2


def GW_invariant(dim, d, l, layer=1):

        """
        Returns the Gromov-Witten invariants for this projective space.
        INPUT: 
            d is the degree
            l is a list of non-negative integers: for example the number 3
            denotes the class h^3.
        OUTPUT:
            The number of degree d curves meeting some general subvarieties in
            this projective space.
        EXAMPLES::
            Computing the number of lines meeting 4 general lines in P^3.
            sage: P = ProjectiveSpace(3)
            sage: P.GW_invariant(1,[2,2,2,2])
            2
            Computing the number of conics meeting 8 general lines in P^3.
            sage: P = ProjectiveSpace(3)
            sage: P.GW_invariant(2,[2]*8)            
            92
        """
        r = dim
        l.sort()
        l.reverse()
        n = len(l)
        for i in l:
            if i > r:
                return 0
            pass
        if sum(l) != r*d + r + d + n - 3:
            return 0
        elif n == 0 or n == 1:
            return 0
        elif d == 0:
            if n == 3 and sum(l) == r:
                return 1
            else:
                return 0
        elif n == 2:
            if d == 1 and l == [r,r]:
                return 1
            else:
                return 0
        else:
            if l[-1] == 0:
                return 0
            elif l[-1] == 1:
                l.pop(-1)
                return d*GW_invariant(dim, d,l)
            elif r == 2:
                if l == [2]*(3*d-1):
                    return kontsevich(d)
                else:
                    return 0
            else:
                l1 = l[-1] - 1
                l2 = 1
                ll = [l2] + [l[i] for i in range(n-2)] + [l1+l[n-2]]
                res = GW_invariant(dim, d,ll)
                S = bipart([l[i] for i in range(n-3)])

                start = res
                pos = 0
                neg = 0
                for s in S:
                    A = s[0]
                    nA = len(A)
                    cA = sum(A)
                    B = s[1]
                    for dA in range(1,d+1):
                        dB = d - dA
                        e = r*dA + r + dA + nA - cA - l1 - l[n-2]
                        if e >= 0 and e <= r:
                            a = GW_invariant(dim, dA,[l1,l[n-2]]+A+[e])
                            b = GW_invariant(dim, dB,[l2,l[n-3]]+B+[r-e])
                            res = res + a*b
                            pos += a*b
                            if layer ==2 :
                                print(f"l' : {l[:n-3]}, A: {A}")
                                print(f'd: {d}, l: {l}, pos: {pos}, dA: {dA}, e: {e}')
                        f = r*dA + r + dA + nA - cA - l1 - l2
                        if f >= 0 and f <= r:
                            x = GW_invariant(dim, dA,[l1,l2]+A+[f])
                            y = GW_invariant(dim, dB,[l[n-2],l[n-3]]+B+[r-f])
                            res = res - x*y
                            neg += x*y
                print(f'd: {d}, l: {l}, start: {start}, res: {res}, pos: {pos}, neg: {neg}')
                return res

print(GW_invariant(3, 2, [3, 3, 3, 2, 2], layer=2))

d: 1, l: [3, 2, 2], start: 1, res: 1, pos: 0, neg: 0
d: 1, l: [3, 2, 2], start: 1, res: 1, pos: 0, neg: 0
d: 2, l: [3, 3, 3, 3], start: 0, res: 0, pos: 1, neg: 1
d: 1, l: [3, 2, 2], start: 1, res: 1, pos: 0, neg: 0
d: 2, l: [3, 3, 3, 2, 2], pos: 1, dA: 1, e: 2
d: 2, l: [3, 3, 3, 2, 2], pos: 1, dA: 1, e: 0
d: 1, l: [3, 2, 2], start: 1, res: 1, pos: 0, neg: 0
d: 1, l: [3, 2, 2], start: 1, res: 1, pos: 0, neg: 0
d: 2, l: [3, 3, 3, 2, 2], pos: 2, dA: 1, e: 2
d: 2, l: [3, 3, 3, 2, 2], start: 0, res: 1, pos: 2, neg: 1
1
